## **STEP1. 랭체인으로 GPT 부르기**

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os

📌 핵심 개념:
- PromptTemplate: 변수를 포함한 프롬프트 정의
- system 메시지: AI의 역할/성격 지정
- human 메시지: 사용자의 실제 입력
- 템플릿을 쓰면 프롬프트 재사용이 쉬워짐

In [2]:
load_dotenv("key.env")

llm=ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
prompt=ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 선생님입니다. 초등학생도 이해할 수 있게 설명해주세요."),
    ("user", "{question}"),
])

#prompt와 llm을 파이프로 연결
chain = prompt | llm

#llm 단독호출
response = chain.invoke("양자역학이 뭐야?")
print(response.content)

좋은 질문이야! 양자역학은 아주아주 작은 세상, 예를 들어 원자나 전자처럼 눈에 보이지 않는 아주 작은 입자들이 어떻게 움직이고 행동하는지를 연구하는 과학이야.

우리가 일상에서 보는 큰 물건들은 보통 뉴턴의 법칙이라는 규칙을 따라 움직여. 하지만 아주 작은 세계에서는 그 규칙이 잘 안 맞아. 그래서 과학자들이 새로운 규칙을 만들었는데, 그게 바로 양자역학이야.

양자역학에서는 작은 입자들이 마치 파도처럼 움직이기도 하고, 동시에 여러 곳에 있을 수도 있어. 그리고 우리가 그 입자를 관찰하면 그때서야 어디에 있는지 알 수 있지.

쉽게 말하면, 양자역학은 아주 작은 세계의 신기한 규칙을 알려주는 과학이라고 생각하면 돼!


In [3]:
!pip install -U langchain-google-genai

  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 724.2/724.2 kB 6.4 MB/s  0:00:00
Using cached rsa-4.9.1-py3-none-any.whl (34 kB)
Using cached pyasn1_modules-0.4.2-py3-none-any.whl (181 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [langchain-google-genai]e-genai]s]


In [5]:
from getpass import getpass

In [6]:
# Gemini API Key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Gemini API Key: ")

print("✅ OpenAI & Gemini API Key 설정 완료!")

✅ OpenAI & Gemini API Key 설정 완료!


| 패키지                   | 역할                                       |
| --------------------- | ---------------------------------------- |
| langchain-core      | **핵심 추상 개념 (Prompt, Runnable, Message)** |
| langchain           | 체인, 메모리, 에이전트 구현                         |
| langchain-openai    | OpenAI 연동                                |
| langchain-community | 서드파티 도구                                  |

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash"
)

response = llm.invoke("안녕?")
print(response.content)

안녕하세요! 반갑습니다. 😊


## **Step 2: 메모리 관리 (대화 기록)**

[핵심 개념]
- ChatMessageHistory: 대화 기록을 저장하는 클래스
- RunnableWithMessageHistory: 메모리를 포함한 체인
- session_id: 대화 세션을 구분하는 ID
- store: 여러 세션의 대화 기록을 저장하는 딕셔너리

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [ ]:
#1. llm
llm = ChatOpenAI(model_name="gpt-4.1-mini")

#2. prompt (history를 끼워 넣는 것이 핵심)
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

#3. 세션별 대화 저장소
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

#4. chain
chain = prompt | llm

#5. memory를 체인에 연결
chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_variable_name="history"
)